# Connections

> Connection management for SSE clients

In [ ]:
#| default_exp core.connections

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import asyncio
from typing import Dict, Optional, Any, Set, Callable
from datetime import datetime
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import Div, Script

## Core Classes

In [ ]:
#| export
class ConnectionState(Enum):
    """States for SSE connections"""
    CONNECTING = "connecting"
    CONNECTED = "connected"
    DISCONNECTED = "disconnected"
    ERROR = "error"
    RECONNECTING = "reconnecting"

In [ ]:
#| export
@dataclass
class SSEConnection:
    """Represents a single SSE connection"""
    connection_id: str
    queue: asyncio.Queue
    connection_type: str = "global"
    state: ConnectionState = ConnectionState.CONNECTING
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: datetime = field(default_factory=datetime.now)
    last_activity: datetime = field(default_factory=datetime.now)
    message_count: int = 0
    
    async def send(
        self,
        data: Any,  # TODO: Add description
        timeout: float = 1.0  # TODO: Add description
    ) -> bool:  # TODO: Add return description
        """Send data through the connection queue.
        
        Args:
            data: Data to send
            timeout: Timeout for the send operation
            
        Returns:
            True if successful, False otherwise
        """
        try:
            await asyncio.wait_for(self.queue.put(data), timeout=timeout)
            self.last_activity = datetime.now()
            self.message_count += 1
            return True
        except (asyncio.TimeoutError, asyncio.QueueFull):
            self.state = ConnectionState.ERROR
            return False
    
    async def heartbeat(
        self
    ) -> str:  # TODO: Add return description
        """Generate a heartbeat message."""
        self.last_activity = datetime.now()
        return f": heartbeat {self.connection_id} {self.last_activity.isoformat()}\n\n"
    
    def close(self):
        """Mark the connection as closed."""
        self.state = ConnectionState.DISCONNECTED
    
    def is_active(
        self
    ) -> bool:  # TODO: Add return description
        """Check if connection is active."""
        return self.state in [ConnectionState.CONNECTED, ConnectionState.CONNECTING]

In [ ]:
#| export
class ConnectionRegistry:
    """Registry to track and manage SSE connections"""
    
    def __init__(
        self,
        debug: bool = False  # TODO: Add description
    ):
        """Initialize the connection registry.
        
        Args:
            debug: Enable debug logging
        """
        self.connections: Dict[str, SSEConnection] = {}
        self.connections_by_type: Dict[str, Set[str]] = {}
        self.lock = asyncio.Lock()
        self.debug = debug
        self._counter = 0
    
    async def add_connection(self,
                            conn_id: Optional[str] = None,  # TODO: Add description
                            conn_type: str = "global",  # TODO: Add description
                            queue_size: int = 100,  # TODO: Add description
                            metadata: Optional[Dict[str, Any]] = None) -> SSEConnection:
        """Add a new connection to the registry.
        
        Args:
            conn_id: Optional connection ID (auto-generated if not provided)
            conn_type: Type of connection (e.g., 'global', 'job', 'user')
            queue_size: Size of the message queue
            metadata: Optional metadata for the connection
            
        Returns:
            The created SSEConnection
        """
        async with self.lock:
            if conn_id is None:
                self._counter += 1
                conn_id = f"{conn_type}_{self._counter}"
            
            queue = asyncio.Queue(maxsize=queue_size)
            connection = SSEConnection(
                connection_id=conn_id,
                queue=queue,
                connection_type=conn_type,
                metadata=metadata or {}
            )
            
            self.connections[conn_id] = connection
            
            # Track by type
            if conn_type not in self.connections_by_type:
                self.connections_by_type[conn_type] = set()
            self.connections_by_type[conn_type].add(conn_id)
            
            connection.state = ConnectionState.CONNECTED
            
            if self.debug:
                print(f"[ConnectionRegistry] Added connection: {conn_id} (type: {conn_type})")
            
            return connection
    
    async def remove_connection(
        self,
        conn_id: str  # TODO: Add description
    ):
        """Remove a connection from the registry.
        
        Args:
            conn_id: Connection ID to remove
        """
        async with self.lock:
            if conn_id in self.connections:
                connection = self.connections[conn_id]
                connection.close()
                
                # Remove from type tracking
                conn_type = connection.connection_type
                if conn_type in self.connections_by_type:
                    self.connections_by_type[conn_type].discard(conn_id)
                    if not self.connections_by_type[conn_type]:
                        del self.connections_by_type[conn_type]
                
                del self.connections[conn_id]
                
                if self.debug:
                    print(f"[ConnectionRegistry] Removed connection: {conn_id}")
    
    def get_connection(
        self,
        conn_id: str  # TODO: Add description
    ) -> Optional[SSEConnection]:  # TODO: Add return description
        """Get a specific connection.
        
        Args:
            conn_id: Connection ID
            
        Returns:
            The connection if found, None otherwise
        """
        return self.connections.get(conn_id)
    
    def get_connections(
        self,
        conn_type: Optional[str] = None  # TODO: Add description
    ) -> list[SSEConnection]:  # TODO: Add return description
        """Get connections, optionally filtered by type.
        
        Args:
            conn_type: Optional connection type to filter by
            
        Returns:
            List of connections
        """
        if conn_type:
            conn_ids = self.connections_by_type.get(conn_type, set())
            return [self.connections[cid] for cid in conn_ids if cid in self.connections]
        return list(self.connections.values())
    
    def get_active_connections(
        self,
        conn_type: Optional[str] = None  # TODO: Add description
    ) -> list[SSEConnection]:  # TODO: Add return description
        """Get active connections.
        
        Args:
            conn_type: Optional connection type to filter by
            
        Returns:
            List of active connections
        """
        connections = self.get_connections(conn_type)
        return [c for c in connections if c.is_active()]
    
    def get_stats(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Get registry statistics.
        
        Returns:
            Dictionary with connection statistics
        """
        active_conns = self.get_active_connections()
        
        return {
            "total": len(self.connections),
            "active": len(active_conns),
            "by_type": {k: len(v) for k, v in self.connections_by_type.items()},
            "message_count": sum(c.message_count for c in self.connections.values())
        }

## Helper Functions

In [ ]:
#| export
def create_sse_element(endpoint: str,
                      element_id: Optional[str] = None,  # Optional element ID
                      swap_strategy: str = "message",  # HTMX swap strategy (message, innerHTML, outerHTML, etc.)
                      hidden: bool = False,  # Whether to hide the element **attrs: Additional attributes for the element
                      **attrs) -> Div:
    "Create an SSE-enabled HTML element."
    sse_attrs = {
        'hx_ext': 'sse',
        'sse_connect': endpoint,
        'sse_swap': swap_strategy
    }
    
    if element_id:
        sse_attrs['id'] = element_id
    
    if hidden:
        sse_attrs['style'] = 'display: none;'
    
    # Merge with additional attributes
    sse_attrs.update(attrs)
    
    return Div(**sse_attrs)

In [ ]:
#| export
def cleanup_sse_on_unload(
) -> Script:  # Script element for cleanup
    "Create a script to clean up SSE connections on page unload."
    cleanup_code = """
    window.addEventListener('beforeunload', function() {
        // Close all SSE connections
        document.querySelectorAll('[sse-connect]').forEach(function(element) {
            var internalData = element['htmx-internal-data'];
            if (internalData && internalData.sseEventSource) {
                internalData.sseEventSource.close();
            }
        });
        
        // Trigger HTMX cleanup
        if (typeof htmx !== 'undefined') {
            htmx.findAll('[sse-connect]').forEach(function(element) {
                htmx.trigger(element, 'htmx:sseClose');
            });
        }
    });
    
    // Also handle page visibility changes
    document.addEventListener('visibilitychange', function() {
        if (document.hidden) {
            // Page is hidden - connections will be suspended by browser
            console.log('[SSE] Page hidden - connections suspended');
        } else {
            // Page is visible - connections may need reconnection
            console.log('[SSE] Page visible - checking connections');
        }
    });
    """
    
    return Script(cleanup_code)

In [ ]:
#| export
def create_reconnection_script(check_interval: int = 5000,
                              max_retries: int = 5,  # Maximum number of reconnection attempts
                              debug: bool = False) -> Script:
    "Create a script for automatic SSE reconnection."
    debug_str = "true" if debug else "false"
    
    reconnect_code = f"""
    (function() {{
        var checkInterval = {check_interval};
        var maxRetries = {max_retries};
        var debug = {debug_str};
        var retryCount = {{}};
        
        function log(message) {{
            if (debug) {{
                console.log('[SSE Reconnect] ' + message);
            }}
        }}
        
        function checkConnection(element) {{
            var internalData = element['htmx-internal-data'];
            if (!internalData || !internalData.sseEventSource) {{
                return;
            }}
            
            var source = internalData.sseEventSource;
            var elementId = element.id || 'unknown';
            
            if (source.readyState === EventSource.CLOSED) {{
                if (!retryCount[elementId]) {{
                    retryCount[elementId] = 0;
                }}
                
                if (retryCount[elementId] < maxRetries) {{
                    retryCount[elementId]++;
                    log('Reconnecting ' + elementId + ' (attempt ' + retryCount[elementId] + ')');
                    htmx.trigger(element, 'htmx:sseReconnect');
                }} else {{
                    log('Max retries reached for ' + elementId);
                }}
            }} else if (source.readyState === EventSource.OPEN) {{
                // Reset retry count on successful connection
                retryCount[elementId] = 0;
            }}
        }}
        
        // Check connections periodically
        setInterval(function() {{
            document.querySelectorAll('[sse-connect]').forEach(checkConnection);
        }}, checkInterval);
        
        // Reset retry counts on visibility change
        document.addEventListener('visibilitychange', function() {{
            if (!document.hidden) {{
                log('Page visible - resetting retry counts');
                retryCount = {{}};
            }}
        }});
    }})();
    """
    
    return Script(reconnect_code)

In [ ]:
#| export
def create_connection_manager_script(registry_endpoint: str = "/sse/connections",
                                    update_interval: int = 10000) -> Script:
    "Create a script to manage and monitor connections."
    manager_code = f"""
    (function() {{
        var registryEndpoint = '{registry_endpoint}';
        var updateInterval = {update_interval};
        
        function updateConnectionStats() {{
            fetch(registryEndpoint)
                .then(response => response.json())
                .then(data => {{
                    // Dispatch custom event with connection stats
                    var event = new CustomEvent('sse:stats', {{
                        detail: data
                    }});
                    document.dispatchEvent(event);
                    
                    // Update any elements with data-sse-stats attribute
                    document.querySelectorAll('[data-sse-stats]').forEach(function(element) {{
                        var statType = element.getAttribute('data-sse-stats');
                        if (data[statType] !== undefined) {{
                            element.textContent = data[statType];
                        }}
                    }});
                }})
                .catch(error => {{
                    console.error('[SSE Manager] Failed to fetch stats:', error);
                }});
        }}
        
        // Initial update
        setTimeout(updateConnectionStats, 100);
        
        // Periodic updates
        setInterval(updateConnectionStats, updateInterval);
        
        // Listen for HTMX SSE events
        document.body.addEventListener('htmx:sseOpen', function(evt) {{
            console.log('[SSE Manager] Connection opened:', evt.detail.elt.id);
            updateConnectionStats();
        }});
        
        document.body.addEventListener('htmx:sseClose', function(evt) {{
            console.log('[SSE Manager] Connection closed:', evt.detail.elt.id);
            updateConnectionStats();
        }});
    }})();
    """
    
    return Script(manager_code)

## Examples

### Basic Connection Management

In [ ]:
# Example: Basic connection setup
registry = ConnectionRegistry(debug=True)

# Add a connection
async def add_user_connection(user_id: str):
    connection = await registry.add_connection(
        conn_id=f"user_{user_id}",
        conn_type="user",
        metadata={"user_id": user_id, "permissions": ["read", "write"]}
    )
    return connection

# Get stats
def get_connection_stats():
    return registry.get_stats()

### Creating SSE Elements

In [ ]:
# Example: Create various SSE elements
from fasthtml.common import Div, Span

# Global updates connection
global_sse = create_sse_element(
    endpoint="/sse/global",
    element_id="global-updates",
    hidden=True
)

# Job-specific updates
job_sse = create_sse_element(
    endpoint="/sse/job/123",
    element_id="job-123-updates",
    swap_strategy="innerHTML"
)

# User notifications
notification_sse = create_sse_element(
    endpoint="/sse/notifications",
    element_id="user-notifications",
    swap_strategy="beforeend",  # Append new notifications
    cls="notification-container"
)

### Complete Page Setup

In [ ]:
# Example: Complete page with connection management
def create_page_with_sse():
    from fasthtml.common import Html, Head, Body, Div, Span, Script
    
    return Html(
        Head(
            # Add necessary scripts
            Script(src="https://unpkg.com/htmx.org"),
            Script(src="https://unpkg.com/htmx-ext-sse"),
            cleanup_sse_on_unload(),
            create_reconnection_script(debug=True),
            create_connection_manager_script()
        ),
        Body(
            # Connection status display
            Div(
                "Active connections: ",
                Span("0", data_sse_stats="active"),
                " / ",
                Span("0", data_sse_stats="total"),
                id="connection-status"
            ),
            
            # Global SSE connection
            create_sse_element(
                endpoint="/sse/global",
                element_id="global-sse",
                hidden=True
            ),
            
            # Content area
            Div(
                id="content",
                cls="container"
            )
        )
    )

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()